In [ ]:
# KITTI Dataset Downloader and Loader for Google Colab
import os
import urllib.request
import zipfile
import subprocess
import numpy as np
import cv2
import matplotlib.pyplot as plt



download_links = {
     'object_detection': 'https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip',
     'object_detection_labels': 'https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip',
     'road_data': 'https://s3.eu-central-1.amazonaws.com/avg-kitti/data_road.zip'
}


for name,url in download_links.items():
    print(f"Downloading {name} dataset...")
    try:
        urllib.request.urlretrieve(url, name)
        print(f"Downloaded {name} successfully.")
    except Exception as e:
        print(f"Failed to download {name}: {e}")


Downloaded object_detection successfully.
Downloaded object_detection_labels successfully.
Downloaded road_data successfully.


In [ ]:
# prompt: write a loop that unzip files from a list of names

for name, url in download_links.items():
    try:
        with zipfile.ZipFile(name, 'r') as zip_ref:
            print(f"Extracting {name}...")
            zip_ref.extractall(f"{name}_extracted")
            print(f"Extracted {name} successfully.")
    except FileNotFoundError:
        print(f"Error: {name} file not found.  Please ensure it was downloaded correctly.")
    except zipfile.BadZipFile:
        print(f"Error: {name} is not a valid zip file. Please check the downloaded file.")
    except Exception as e:
        print(f"An error occurred while extracting {name}: {e}")


Extracting object_detection...
Extracted object_detection successfully.
Extracting object_detection_labels...
Extracted object_detection_labels successfully.
Extracting road_data...
Extracted road_data successfully.


In [ ]:
#data praperation
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
lables_path = 'object_detection_labels_extracted/training/label_2'
images_path = 'object_detection_extracted/training/image_2'

CLASS_MAPPING = {
    'Car': 0,
    'Van': 1,
    'Truck': 2,
    'Pedestrian': 3,
    'Person_sitting': 4,
    'Cyclist': 5,
    'Tram': 6,
    'Misc': 7,
    'DontCare': 8
}

In [ ]:
def convert_kitti_to_yolo(kitti_label_path, img_width, img_height):
    """Convert a KITTI label file to YOLO format with robust error handling and logging."""
    yolo_labels = []
    skipped = 0

    try:
        with open(kitti_label_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = line.strip().split()
            if len(data) < 8:
                print(f"⚠️ Skipping malformed line in {kitti_label_path}: {line.strip()}")
                continue

            class_name = data[0]
            if class_name not in CLASS_MAPPING:
                print(f"❌ Unknown class '{class_name}' in {kitti_label_path}")
                continue

            class_id = CLASS_MAPPING[class_name]
            if class_id == 8:
                skipped += 1
                continue  # Skip 'DontCare'

            try:
                x_min, y_min = float(data[4]), float(data[5])
                x_max, y_max = float(data[6]), float(data[7])
            except ValueError:
                print(f"⚠️ Non-numeric box in {kitti_label_path}: {line.strip()}")
                continue

            # YOLO normalized bbox
            center_x = ((x_min + x_max) / 2) / img_width
            center_y = ((y_min + y_max) / 2) / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height

            yolo_labels.append(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}")

    except Exception as e:
        print(f"💥 Error processing {kitti_label_path}: {e}")
        return []

    if not yolo_labels:
        print(f"⚠️ No valid labels in {kitti_label_path} (skipped {skipped} objects)")

    return yolo_labels


In [ ]:
import os
import random
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO
from PIL import Image
import cv2
import glob
from tqdm import tqdm

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# prompt: create folder in a path

import os

def create_folder(path):
  """Creates a folder at the specified path if it doesn't exist.

  Args:
      path: The path to the folder.
  """
  try:
    os.makedirs(path, exist_ok=True)  # exist_ok prevents error if folder exists
    print(f"Folder created successfully at: {path}")
  except OSError as e:
    print(f"Error creating folder: {e}")

# Example usage (replace with your desired path):
create_folder("kitti/kitti_train_images")
create_folder("kitti/kitti_train_labels")
create_folder("kitti/kitti_val_images")
create_folder("kitti/kitti_val_labels")
create_folder("kitti/train")
create_folder("kitti/validate")


Folder created successfully at: kitti/kitti_train_images
Folder created successfully at: kitti/kitti_train_labels
Folder created successfully at: kitti/kitti_val_images
Folder created successfully at: kitti/kitti_val_labels
Folder created successfully at: kitti/train
Folder created successfully at: kitti/validate


In [ ]:
# Update the `prepare_dataset` function with print statements for debugging
def prepare_dataset(images_dir, labels_dir, train_ratio=0.8):

    """Prepare KITTI dataset splitting into train/val and converting to YOLO format"""

    # Look for both .png and .jpg images
    image_files = sorted(glob.glob(os.path.join(images_dir, '*.png')) + glob.glob(os.path.join(images_dir, '*.jpg')))

    # Get all label files
    label_files = sorted(glob.glob(os.path.join(labels_dir, '*.txt')))

    # Extract the IDs (filenames without extension)
    image_ids = [os.path.splitext(os.path.basename(f))[0] for f in image_files]
    label_ids = [os.path.splitext(os.path.basename(f))[0] for f in label_files]

    # Find IDs that have both image and label files
    valid_ids = list(set(image_ids) & set(label_ids))

    # Filter image and label files
    image_files = [f for f in image_files if os.path.splitext(os.path.basename(f))[0] in valid_ids]
    label_files = [f for f in label_files if os.path.splitext(os.path.basename(f))[0] in valid_ids]

    # Shuffle and split
    temp = list(zip(image_files, label_files))  # Zip image and label files together
    random.shuffle(temp)
    image_files, label_files = zip(*temp)  # Unzip the shuffled list
    split_idx = int(len(image_files) * train_ratio)
    train_images = image_files[:split_idx]
    val_images = image_files[split_idx:]
    train_labels = label_files[:split_idx]
    val_labels = label_files[split_idx:]

    print(f"Total valid images: {len(image_files)}")
    print(f"Training images: {len(train_images)}")
    print(f"Validation images: {len(val_images)}")

    # Process training set
    for img_path, label_path in tqdm(zip(train_images, train_labels), desc="Processing training set"):
        img_basename = os.path.basename(img_path)
        img_id = os.path.splitext(img_basename)[0]

        # Get image dimensions
        img = Image.open(img_path)
        img_width, img_height = img.size

        # Convert label to YOLO format
        yolo_labels = convert_kitti_to_yolo(label_path, img_width, img_height)

        if yolo_labels:  # Only include if we have valid labels
            # Save image to train folder
            shutil.copy(img_path, os.path.join("/content/kitti/kitti_train_images", img_basename))

            # Save YOLO label
            with open(os.path.join("/content/kitti/kitti_train_labels", f"{img_id}.txt"), 'w') as f:
                f.write('\n'.join(yolo_labels))

            # Print or log the number of labels for debugging
            print(f"Image {img_id}: Found {len(yolo_labels)} labels")
        else:
            print(f"Image {img_id}: **WARNING: No valid labels found in {label_path}**")  # Debug: No valid labels

    # Process validation set (similar process)
    for img_path, label_path in tqdm(zip(val_images, val_labels), desc="Processing validation set"):
        img_basename = os.path.basename(img_path)
        img_id = os.path.splitext(img_basename)[0]

        # Get image dimensions
        img = Image.open(img_path)
        img_width, img_height = img.size

        # Convert label to YOLO format
        yolo_labels = convert_kitti_to_yolo(label_path, img_width, img_height)

        if yolo_labels:  # Only include if we have valid labels
            # Save image to val folder
            shutil.copy(img_path, os.path.join("/content/kitti/kitti_val_images", img_basename))


            # Save YOLO label
            with open(os.path.join("/content/kitti/kitti_val_labels", f"{img_id}.txt"), 'w') as f:
                f.write('\n'.join(yolo_labels))

            # Print or log the number of labels for debugging
            print(f"Image {img_id}: Found {len(yolo_labels)} labels")
        else:
            print(f"Image {img_id}: **WARNING: No valid labels found in {label_path}**")  # Debug: No valid labels

In [ ]:
prepare_dataset('/content/object_detection_extracted/training/image_2', '/content/object_detection_labels_extracted/training/label_2')

Total valid images: 7481
Training images: 5984
Validation images: 1497


Processing training set: 5it [00:00, 45.35it/s]

Image 002984: Found 8 labels
Image 004446: Found 3 labels
Image 003706: Found 10 labels
Image 005345: Found 5 labels
Image 003637: Found 3 labels
Image 006402: Found 5 labels
Image 004407: Found 3 labels
Image 002364: Found 5 labels
Image 004531: Found 2 labels
Image 004604: Found 12 labels
Image 001889: Found 2 labels
Image 000750: Found 5 labels
Image 003670: Found 1 labels


Processing training set: 15it [00:00, 76.16it/s]

Image 004511: Found 10 labels
Image 005364: Found 3 labels
Image 004640: Found 10 labels
Image 001070: Found 11 labels
Image 005509: Found 2 labels
Image 006824: Found 5 labels


...

Image 006152: Found 1 labels
Image 006091: Found 2 labels
Image 006929: Found 11 labels
Image 001816: Found 6 labels
Image 007270: Found 2 labels
Image 001013: Found 11 labels
Image 000044: Found 2 labels
Image 002437: Found 5 labels
Image 006052: Found 9 labels
Image 005250: Found 9 labels
Image 001036: Found 1 labels
Image 000055: Found 5 labels
Image 006886: Found 10 labels
Image 005759: Found 5 labels
Image 000969: Found 4 labels
Image 004210: Found 4 labels
Image 006781: Found 8 labels
Image 002433: Found 7 labels
Image 000027: Found 2 labels
Image 001953: Found 3 labels
Image 004193: Found 14 labels
Image 002945: Found 4 labels
Image 007185: Found 10 labels
Image 006253: Found 6 labels
Image 006761: Found 8 labels
Image 003464: Found 16 labels
Image 004510: Found 6 labels
Image 000318: Found 6 labels
Image 003202: Found 1 labels


Processing validation set: 1470it [00:11, 109.51it/s]

Image 001986: Found 7 labels
Image 000874: Found 17 labels
Image 004471: Found 4 labels
Image 007251: Found 1 labels
Image 005990: Found 2 labels
Image 003226: Found 3 labels
Image 003206: Found 12 labels
Image 003742: Found 5 labels
Image 004271: Found 5 labels
Image 006941: Found 1 labels
Image 004081: Found 4 labels
Image 003396: Found 8 labels
Image 006251: Found 14 labels
Image 002186: Found 1 labels
Image 007000: Found 9 labels
Image 004693: Found 4 labels
Image 000809: Found 11 labels
Image 006328: Found 8 labels
Image 003965: Found 4 labels
Image 000237: Found 3 labels
Image 001943: Found 3 labels
Image 005651: Found 3 labels
Image 006571: Found 6 labels
Image 002707: Found 19 labels
Image 006881: Found 6 labels
Image 006529: Found 2 labels
Image 003745: Found 6 labels


Processing validation set: 1497it [00:11, 125.03it/s]

Image 003480: Found 8 labels
Image 003738: Found 2 labels
Image 004057: Found 6 labels
Image 005254: Found 2 labels
Image 003188: Found 1 labels
Image 001908: Found 16 labels
Image 003295: Found 1 labels
Image 004118: Found 8 labels
Image 007272: Found 1 labels
Image 001193: Found 8 labels
Image 004402: Found 5 labels
Image 005541: Found 6 labels
Image 007009: Found 1 labels
Image 006077: Found 11 labels
Image 005508: Found 8 labels
Image 001454: Found 6 labels
Image 001792: Found 5 labels


In [ ]:
print(len(os.listdir('/content/kitti/kitti_train_images')))
print(len(os.listdir('/content/kitti/kitti_train_labels')))

print(len(os.listdir('/content/kitti/kitti_val_images')))
print(len(os.listdir('/content/kitti/kitti_val_labels')))

5984
5984
1497
1497


In [ ]:
# prompt: create a new folder with copied contents from 2 different folders

import os
import shutil

def create_new_folder_with_copied_contents(source_folder1, source_folder2, destination_folder):
    """
    Creates a new folder and copies contents from two source folders into it.

    Args:
        source_folder1: Path to the first source folder.
        source_folder2: Path to the second source folder.
        destination_folder: Path to the new destination folder.
    """
    try:
        # Create the destination folder if it doesn't exist
        os.makedirs(destination_folder, exist_ok=True)

        # Copy contents from the first source folder
        for item in os.listdir(source_folder1):
            source_item = os.path.join(source_folder1, item)
            destination_item = os.path.join(destination_folder, item)
            if os.path.isfile(source_item):
                shutil.copy2(source_item, destination_item)  # copy2 preserves metadata
            elif os.path.isdir(source_item):
                shutil.copytree(source_item, destination_item)


        # Copy contents from the second source folder
        for item in os.listdir(source_folder2):
          source_item = os.path.join(source_folder2, item)
          destination_item = os.path.join(destination_folder, item)
          if os.path.isfile(source_item):
              shutil.copy2(source_item, destination_item) # copy2 preserves metadata
          elif os.path.isdir(source_item):
              shutil.copytree(source_item, destination_item)

        print(f"Successfully copied contents from '{source_folder1}' and '{source_folder2}' to '{destination_folder}'")

    except FileNotFoundError:
        print(f"Error: One or both source folders not found.")
    except OSError as e:
        print(f"Error copying files: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage
source_folder1 = "/content/kitti/kitti_val_images"  # Replace with the actual path
source_folder2 = "/content/kitti/kitti_val_labels"   # Replace with the actual path
destination_folder = "/content/yolov5/kitti/validate"  # Replace with the desired path
create_new_folder_with_copied_contents(source_folder1, source_folder2, destination_folder)

source_folder3 = "/content/kitti/kitti_train_images"  # Replace with the actual path
source_folder4 = "/content/kitti/kitti_train_labels"   # Replace with the actual path
destination_folder2 = "/content/yolov5/kitti/train"  # Replace with the desired path
create_new_folder_with_copied_contents(source_folder3, source_folder4, destination_folder2)


Successfully copied contents from '/content/kitti/kitti_val_images' and '/content/kitti/kitti_val_labels' to '/content/yolov5/kitti/validate'
Successfully copied contents from '/content/kitti/kitti_train_images' and '/content/kitti/kitti_train_labels' to '/content/yolov5/kitti/train'


In [ ]:
# Create dataset YAML file with explicit paths for YOLOv5
yaml_content = """
# KITTI dataset config for YOLOv5 with explicit paths

path: ./kitti_data  # root directory

# Train/val paths
train: /content/yolov5/kitti/train  # absolute path to train images
val: /content/yolov5/kitti/validate  # absolute path to val images

# Classes
nc: 9  # Define number of classes explicitly
names: ["Car", "Van", "Truck", "Pedestrian", "Person_sitting", "Cyclist", "Tram", "Misc", "DontCare"]
"""

with open("kitti_custom_paths.yaml", "w") as f:
    f.write(yaml_content)

# Clone YOLOv5 repository to access additional utilities
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

# Download a pre-trained YOLOv5 model - using small variant (YOLOv5s)
model = YOLO('yolov5m.pt')  # pre-trained on COCO dataset


fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
PRO TIP 💡 Replace 'model=yolov5m.pt' with new 'model=yolov5mu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 48.2M/48.2M [00:01<00:00, 33.4MB/s]


In [ ]:
model.info(verbose=True)

YOLOv5m summary: 197 layers, 25,111,456 parameters, 0 gradients, 64.6 GFLOPs


(197, 25111456, 0, 64.606464)

In [ ]:
# prompt: delete a folder path

import shutil

# Specify the folder path to delete
folder_path_to_delete = "/content/yolov5/kitti/train"  # Replace with the actual path

try:
    shutil.rmtree(folder_path_to_delete)
    print(f"Folder '{folder_path_to_delete}' and its contents deleted successfully.")
except FileNotFoundError:
    print(f"Error: Folder '{folder_path_to_delete}' not found.")
except OSError as e:
    print(f"Error deleting folder: {e}")


Folder '/content/yolov5/kitti/train' and its contents deleted successfully.


In [ ]:
# Get the current working directory
cwd = os.getcwd()
yaml_path = os.path.join(cwd, "kitti_custom_paths.yaml")

# Set training parameters
epochs = 2
batch_size = 4
img_size = 416


# Then use this path in your training call
results = model.train(
    data= "/content/kitti_custom_paths.yaml",  # Use absolute path
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    patience=5,
    save=True,
    freeze = 20,
    device=0 if torch.cuda.is_available() else 'cpu',
    verbose = True
)

Ultralytics 8.3.126 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/kitti_custom_paths.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=20, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov5m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5, perspective=0.0, plots=True, pose=12.0, pretrained=True

100%|██████████| 755k/755k [00:00<00:00, 3.09MB/s]

Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1      5280  ultralytics.nn.modules.conv.Conv             [3, 48, 6, 2, 2]              
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2     65280  ultralytics.nn.modules.block.C3              [96, 96, 2]                   
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    444672  ultralytics.nn.modules.block.C3              [192, 192, 4]                 
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              


  6                  -1  6   2512896  ultralytics.nn.modules.block.C3              [384, 384, 6]                 
  7                  -1  1   2655744  ultralytics.nn.modules.conv.Conv             [384, 768, 3, 2]              
  8                  -1  2   4134912  ultralytics.nn.modules.block.C3              [768, 768, 2]                 
  9                  -1  1   1476864  ultralytics.nn.modules.block.SPPF            [768, 768, 5]                 
 10                  -1  1    295680  ultralytics.nn.modules.conv.Conv             [768, 384, 1, 1]              
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  2   1182720  ultralytics.nn.modules.block.C3              [768, 384, 2, False]          
 14                  -1  1     74112  ultralytics.nn.modules.conv.Conv             [384,

train: Scanning /content/yolov5/kitti/train... 5984 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5984/5984 [00:29<00:00, 200.87it/s]


...

        2/2         0G      1.254     0.9041      1.006         55        416: 100%|██████████| 1496/1496 [29:48<00:00,  1.20s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 188/188 [02:07<00:00,  1.48it/s]


                   all       1497       8116      0.573      0.472      0.498      0.282

2 epochs completed in 1.084 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 50.4MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 50.4MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.126 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLOv5m summary (fused): 106 layers, 25,050,427 parameters, 0 gradients, 64.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 188/188 [01:58<00:00,  1.58it/s]


                   all       1497       8116      0.573      0.472      0.498      0.282
                   Car       1348       5718      0.692      0.782      0.801      0.505
                   Van        427        572      0.578      0.462      0.508       0.33
                 Truck        212        224       0.51      0.647      0.619      0.409
            Pedestrian        346        926      0.771      0.509      0.581      0.303
        Person_sitting         16         37      0.384      0.351      0.343      0.146
               Cyclist        210        307      0.612      0.195      0.303      0.134
                  Tram         73        120      0.333      0.625      0.521      0.247
                  Misc        165        212      0.706      0.208      0.308      0.181
Speed: 0.1ms preprocess, 62.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train2


AttributeError: 'DetMetrics' object has no attribute 't_total'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP).

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class.
        names (dict): A dictionary of class names.
        box (Metric): An instance of the Metric class for storing detection results.
        speed (dict): A dictionary for storing execution times of different parts of the detection process.
        task (str): The task type, set to 'detect'.
    